### Required User Inputs

In [1]:
# Imports
import arcpy
import pandas as pd

# Environment settings
arcpy.env.workspace = 'E:/My Drive/ADHUS_FAU/fauhsresearch/student_output/citations/data/fauhs_citations.gdb'
arcpy.env.overwriteOutput = True

# Required inputs
path = 'E:/My Drive/ADHUS_FAU/fauhsresearch/student_output/citations/data'
workbook = '/inputs/Geography of Citations.xlsx'

### Data Wrangling of "Geography of Citations" Spreadsheet

In [2]:
# Get all worksheets as dict of dfs
workbook_dict = pd.read_excel(f'{path}{workbook}', sheet_name=None)

# Consolidate into a single DataFrame
df = pd.concat(workbook_dict, ignore_index=True) 

print(f'{df.shape[0]} rows by {df.shape[1]} columns')
#df.head()

1540 rows by 32 columns


In [3]:
# Add columns that uniquely represent the location of each citation
df['STATE_CITING_PAPER'] = df['CITED_BY_STATE'] + ', ' + df['CITED_BY_TITLE']
df['COUNTRY_CITING_PAPER'] = df['CITED_BY_COUNTRY'] + ', ' + df['CITED_BY_TITLE']

print(f'{df.shape[0]} rows by {df.shape[1]} columns')
#df.head()

1540 rows by 34 columns


In [4]:
# Split the df to places inside and outside USA
usa_df = df.loc[df['CITED_BY_COUNTRY'] == 'United States']
world_df = df.loc[df['CITED_BY_COUNTRY'] != 'United States']

print(f'Inside USA: {usa_df.shape[0]} records\nOutside USA: {world_df.shape[0]} records')

Inside USA: 810 records
Outside USA: 730 records


In [5]:
# Remove double counting where same citing paper has same location multiple times
usa_df = usa_df.drop_duplicates(['STATE_CITING_PAPER'])
world_df = world_df.drop_duplicates(['COUNTRY_CITING_PAPER'])

print(f'Inside USA: {usa_df.shape[0]} records after duplicates removed \
\nOutside USA: {world_df.shape[0]} records after duplicates removed')

Inside USA: 790 records after duplicates removed 
Outside USA: 580 records after duplicates removed


In [6]:
# Combine results of data wrangling
df = pd.concat([world_df, usa_df])

print(f'{df.shape[0]} rows by {df.shape[1]} columns')
df.head()

1370 rows by 34 columns


,STUDENT,JOURNAL_TITLE,PUBLICATION_TITLE,PUBLICATION_DOI,CITED_BY_STATE,CITED_BY_COUNTRY,CITED_BY_TITLE,CITED_BY_LINK,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,STATE_CITING_PAPER,COUNTRY_CITING_PAPER
5,"Sousa, John",New England Journal of Medicine,Increasing Incidence of the Neonatal Abstinenc...,10.1056/NEJMsa1500439,Tehran,Iran,Development and validation of the Neonatal Abs...,https://www.tandfonline.com/doi/full/10.1080/1...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Tehran, Development and validation of the Neon...","Iran, Development and validation of the Neonat..."
9,"Sousa, John",New England Journal of Medicine,Increasing Incidence of the Neonatal Abstinenc...,10.1056/NEJMsa1500439,Ontario,Canada,Maternal and iatrogenic neonatal opioid withdr...,https://onlinelibrary.wiley.com/doi/epdf/10.10...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Ontario, Maternal and iatrogenic neonatal opio...","Canada, Maternal and iatrogenic neonatal opioi..."
31,"Sousa, John",New England Journal of Medicine,Increasing Incidence of the Neonatal Abstinenc...,10.1056/NEJMsa1500439,Saskatchewan,Canada,The iatrogenic opioid crisis: An example of ‘i...,https://onlinelibrary.wiley.com/doi/10.1111/je...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Saskatchewan, The iatrogenic opioid crisis: An...","Canada, The iatrogenic opioid crisis: An examp..."
33,"Sousa, John",New England Journal of Medicine,Increasing Incidence of the Neonatal Abstinenc...,10.1056/NEJMsa1500439,Nottingham,United Kingdom,The iatrogenic opioid crisis: An example of ‘i...,https://onlinelibrary.wiley.com/doi/10.1111/je...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Nottingham, The iatrogenic opioid crisis: An e...","United Kingdom, The iatrogenic opioid crisis: ..."
36,"Sousa, John",New England Journal of Medicine,Increasing Incidence of the Neonatal Abstinenc...,10.1056/NEJMsa1500439,London,United Kingdom,Cost of neonatal abstinence syndrome: an econo...,https://fn.bmj.com/content/106/5/494,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"London , Cost of neonatal abstinence syndrome:...","United Kingdom, Cost of neonatal abstinence sy..."


### Tracker of Student Publications Included in this Analysis

In [7]:
# Track which student publications are included in this analysis
tracker = pd.Series(df['PUBLICATION_DOI'].unique(), name='DOI')
print(tracker)

_ = tracker.to_excel(f'{path}/outputs/student_publication_tracker.xlsx')

0                                 10.1056/NEJMsa1500439
1                                  10.1642/AUK-17-216.1
2                            10.1038/s41598-018-29605-1
3                             10.1007/978-3-319-96634-2
4            https://doi.org/10.1016/j.bspc.2020.102249
5                             10.18632/oncotarget.15174
6      https://doi.org/10.1016/j.neuropharm.2017.09.020
7                       10.1016/j.expneurol.2018.12.001
8                                        PMID: 30794362
9              https://doi.org/10.1189/jlb.3A0214-114RR
10                https://doi.org/10.3892/ijo.2017.4144
11            https://doi.org/10.1007/s12033-013-9717-x
12             https://doi.org/10.1021/acs.jcim.9b01212
13            https://doi.org/10.1038/s41386-018-0301-8
14           https://doi.org/10.1103/PhysRevC.92.014624
15        https://doi.org/10.1080/01677063.2018.1500571
16         https://doi.org/10.1021/acschemneuro.8b00577
17            https://doi.org/10.1016/j.cca.2019

### Student Publications Cited by States in the USA

In [8]:
# Get "cited by" count for each unique state in the USA
usa_states_aggregation = usa_df.groupby(['CITED_BY_STATE'])['CITED_BY_STATE'].count()
print(usa_states_aggregation)

# Write output (join table) to file
_ = usa_states_aggregation.to_csv(f'{path}/outputs/usa_states_citation_counts.csv',
                                  index_label=['PLACE_NAME', 'CITED_BY'])

CITED_BY_STATE
Alabama               4
Arizona               8
Arkansas              4
California           41
Colorado             16
Connecticut          13
Delaware              4
Florida             111
Georgia              13
Hawaii                4
Illinois             21
Indiana              15
Iowa                  5
Kansas                2
Kentucky             17
Louisiana             2
Maine                 4
Maryland             31
Massachusetts        59
Michigan             10
Minnesota            13
Mississippi           3
Missouri             12
Montana               1
Nebraska              3
Nevada                2
New Hampshire         8
New Jersey            6
New Mexico            4
New York             41
North Carolina       25
North Dakota          6
Ohio                 47
Oklahoma              3
Oregon                4
Pennsylvania         75
Rhode Island          3
South Carolina        9
South Dakota          1
Tennessee            32
Texas                42
U

### Student Publications Cited by World Countries

In [9]:
# Get "cited by" count for each unique country
countries_aggregation = df.groupby(['CITED_BY_COUNTRY'])['CITED_BY_COUNTRY'].count()
print(countries_aggregation)

# Write output (join table) to file
_ = countries_aggregation.to_csv(f'{path}/outputs/countries_citation_counts.csv',
                                 index_label=['PLACE_NAME', 'CITED_BY'])

CITED_BY_COUNTRY
Argentina          3
Australia         31
Austria            5
Bangladesh         4
Belgium           11
                ... 
United States    790
Uruguay            1
Venezuela          2
Vietnam            1
Yemen              1
Name: CITED_BY_COUNTRY, Length: 78, dtype: int64


### Associate "Cited by" Counts with Geographic Features

In [10]:
# Copy USA state polygons to a new feature class
usa_states_geom = arcpy.management.CopyFeatures('usa_states_generalized', 'usa_states_geom')

# Copy country polygons to a new feature class
countries_geom = arcpy.management.CopyFeatures('world_countries_generalized', 'countries_geom')

In [11]:
# Join the "cited by" counts to USA state polygons
_ = arcpy.management.JoinField(usa_states_geom, 'STATE_NAME',
                               f'{path}/outputs/usa_states_citation_counts.csv', 'PLACE_NAME')

# Join the "cited by" counts to country polygons
_ = arcpy.management.JoinField(countries_geom, 'COUNTRY',
                               f'{path}/outputs/countries_citation_counts.csv', 'PLACE_NAME')

In [12]:
# Combine USA states and world countries into a single dataset
geography_of_citations = arcpy.management.Merge([usa_states_geom, countries_geom],
                                                'geography_of_citations')

### Formatting for Map Products

In [13]:
# Consolidate the names of states and countries into a single field
_ = arcpy.management.CalculateField(geography_of_citations, 'PLACE_NAME',
                                    '!STATE_NAME! if !PLACE_NAME! is None else !PLACE_NAME!')

_ = arcpy.management.CalculateField(geography_of_citations, 'PLACE_NAME',
                                    '!COUNTRY! if !PLACE_NAME! is None else !PLACE_NAME!')

# Replace null "cited by" values with zeros
_ = arcpy.management.CalculateField(geography_of_citations, 'CITED_BY',
                                    '0 if !CITED_BY! is None else !CITED_BY!')

In [14]:
# Get list of fields to be dropped (Only nonrequired fields may be deleted)
drop_fields = [field.name for field in arcpy.ListFields(geography_of_citations)
               if not field.required and field.name not in ['PLACE_NAME', 'CITED_BY']]

# Delete unnecessary fields for better web map formatting
_ = arcpy.management.DeleteField(geography_of_citations, drop_fields)